In [1]:
############## DungNB 0.900 ##############

In [2]:
!pip install ../input/cassava-packages/timm-0.3.1-py3-none-any.whl
!pip install ../input/cassava-packages/vision_transformer_pytorch-1.0.2-py2.py3-none-any.whl
!cp -r ../input/cassava-packages/pretrainedmodels-0.7.4-py37hc8dfbb8_0.xyz ./pretrainedmodels-0.7.4-py37hc8dfbb8_0.tar.bz2
!conda install ./pretrainedmodels-0.7.4-py37hc8dfbb8_0.tar.bz2
!rm -rf ./pretrainedmodels-0.7.4-py37hc8dfbb8_0.tar.bz2

Processing /kaggle/input/cassava-packages/timm-0.3.1-py3-none-any.whl
Processing /kaggle/input/cassava-packages/vision_transformer_pytorch-1.0.2-py2.py3-none-any.whl

######################################################################## | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
import os
import numpy as np 
import cv2
import pandas as pd
import pretrainedmodels
import timm
from vision_transformer_pytorch import VisionTransformer
from tqdm import tqdm
from albumentations import Resize, Normalize, Compose, CenterCrop
from albumentations.pytorch import ToTensorV2
import random
import pickle
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
from torch.utils.data import Dataset, DataLoader

from lightgbm import LGBMClassifier
import lightgbm

import warnings
warnings.filterwarnings("ignore")

cv2.setNumThreads(0)
cv2.ocl.setUseOpenCL(False)

In [4]:
TEST_DIR = "../input/cassava-leaf-disease-classification/test_images"
TEST_DF = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
CHECKPOINT_DIR = '../input/cassava-checkpoints'
WORKERS = 2

In [5]:
def seed_everything(seed=123):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 123
seed_everything(SEED)

In [6]:
def save_dict(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_dict(name):
    with open(name, 'rb') as f:
        return pickle.load(f)

In [7]:
class SeResnext(nn.Module):
    def __init__(self, backbone, pretrained, num_classes, in_features):
        super(SeResnext, self).__init__()
        if pretrained:
            self.base = pretrainedmodels.__dict__[backbone](pretrained='imagenet')
        else:
            self.base = pretrainedmodels.__dict__[backbone](pretrained=None)
        self.base.avg_pool = nn.AdaptiveAvgPool2d(output_size=1)
        self.base.last_linear = nn.Identity()
        self.fc1 = nn.Linear(in_features, 2048, bias=True)
        self.fc2 = nn.Linear(2048, 1024, bias=True)
        self.last_linear = nn.Linear(1024, num_classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.base(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_linear(x)
        return x

class ENet(nn.Module):
    def __init__(self, backbone, pretrained, num_classes, in_features):
        super(ENet, self).__init__()
        self.base = timm.create_model(backbone, pretrained=pretrained, num_classes=0)
        self.fc1 = nn.Linear(in_features, 2048, bias=True)
        self.fc2 = nn.Linear(2048, 1024, bias=True)
        self.last_linear = nn.Linear(1024, num_classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.base(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_linear(x)
        return x

class IRV2(nn.Module):
    def __init__(self, backbone, pretrained, num_classes, in_features):
        super(IRV2, self).__init__()
        if pretrained:
            self.base = pretrainedmodels.__dict__[backbone](pretrained='imagenet')
        else:
            self.base = pretrainedmodels.__dict__[backbone](pretrained=None)
        self.base.avg_pool = nn.AdaptiveAvgPool2d(output_size=1)
        self.base.last_linear = nn.Identity()
        self.fc1 = nn.Linear(in_features, 2048, bias=True)
        self.fc2 = nn.Linear(2048, 1024, bias=True)
        self.last_linear = nn.Linear(1024, num_classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.base(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_linear(x)
        return x

class InceptionV4(nn.Module):
    def __init__(self, backbone, pretrained, num_classes, in_features):
        super(InceptionV4, self).__init__()
        if pretrained:
            self.base = pretrainedmodels.__dict__[backbone](pretrained='imagenet')
        else:
            self.base = pretrainedmodels.__dict__[backbone](pretrained=None)
        self.base.avg_pool = nn.AdaptiveAvgPool2d(output_size=1)
        self.base.last_linear = nn.Identity()
        self.fc1 = nn.Linear(in_features, 2048, bias=True)
        self.fc2 = nn.Linear(2048, 1024, bias=True)
        self.last_linear = nn.Linear(1024, num_classes, bias=True)

    @autocast()
    def forward(self, x):
        x = self.base(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_linear(x)
        return x

class ViT(nn.Module):
    def __init__(self, backbone, pretrained, num_classes):
        super(ViT, self).__init__()
        if pretrained:
            self.base = VisionTransformer.from_pretrained(backbone, num_classes=num_classes)
        else:
            self.base = VisionTransformer.from_name(backbone, num_classes=num_classes) 
    @autocast()
    def forward(self, x):
        x = self.base(x)
        return x
    
def make_model(backbone, pretrained, num_classes):
    if backbone == 'eb0':
        return ENet('tf_efficientnet_b0_ns', pretrained, num_classes, in_features=1280)
    elif backbone == 'eb1':
        return ENet('tf_efficientnet_b1_ns', pretrained, num_classes, in_features=1280)
    elif backbone == 'eb2':
        return ENet('tf_efficientnet_b2_ns', pretrained, num_classes, in_features=1408)
    elif backbone == 'eb3':
        return ENet('tf_efficientnet_b3_ns', pretrained, num_classes, in_features=1536)
    elif backbone == 'eb4':
        return ENet('tf_efficientnet_b4_ns', pretrained, num_classes, in_features=1792)
    elif backbone == 'eb5':
        return ENet('tf_efficientnet_b5_ns', pretrained, num_classes, in_features=2048)
    elif backbone == 'eb6':
        return ENet('tf_efficientnet_b6_ns', pretrained, num_classes, in_features=2304)
    elif backbone == 'eb7':
        return ENet('tf_efficientnet_b7_ns', pretrained, num_classes, in_features=2560)
    elif backbone == 'sr101':
        return SeResnext('se_resnext101_32x4d', pretrained, num_classes, in_features=2048)
    elif backbone == 'iv4':
        return InceptionV4('inceptionv4', pretrained, num_classes, in_features=1536)
    elif backbone == 'irv2':
        return IRV2('inceptionresnetv2', pretrained, num_classes, in_features=1536)
    elif backbone == 'vitb16':
        return ViT('ViT-B_16', pretrained, num_classes)
    else:
        raise ValueError('!!! MODELNAME !!!')
        
class CNNLV2Model(nn.Module):
    def __init__(self, num_classes, num_channels):
        super(CNNLV2Model, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(num_channels, 256, kernel_size=(1,3), stride=1, padding=0),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 512, kernel_size=(5,1), stride=1, padding=0),
            nn.ReLU(inplace=True),
        )
        self.fc1 = nn.Linear(512, 1024, bias=True)
        self.fc2 = nn.Linear(1024, 1024, bias=True)
        self.last_linear = nn.Linear(1024, num_classes, bias=True)

    def forward(self, x):
        x = self.conv1(x).view(-1, 512)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.last_linear(x)
        return x

In [8]:
class CLDTestDataset(Dataset):
    def __init__(self, df, root_dir, image_size):
        super(CLDTestDataset,self).__init__()
        self.df = df.reset_index(drop=True)
        self.root_dir = root_dir
        self.image_size = image_size
        self.transform = Compose([
            Resize(self.image_size, self.image_size),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        image_id = self.df.loc[index, 'image_id']
        image_path = os.path.join(self.root_dir, image_id)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        height, width = image.shape[0:2]
        crop_size = int(0.8*min(height, width))
        x1 = (width - crop_size)//2
        y1 = (height - crop_size)//2
        image_cropped = image[y1:y1+crop_size, x1:x1+crop_size, :]
        image_cropped = self.transform(image=image_cropped)['image']
        
        image = self.transform(image=image)['image']
        return image, image_cropped, image_id

In [9]:
def load_checkpoint(checkpoint_dir, backbone='eb6', image_size=512, fold=0):
    model = make_model(backbone=backbone, pretrained=False, num_classes=5)
    model.cuda()
    CHECKPOINT = '{}/{}_{}_fold{}.pth'.format(checkpoint_dir, backbone, image_size, fold)
    model.load_state_dict(torch.load(CHECKPOINT))
    model.eval()
    return model

def load_cnn_lv2_checkpoint(checkpoint_dir, num_models=3, fold=0):
    model = CNNLV2Model(num_classes=5, num_channels=num_models)
    model.cuda()
    CHECKPOINT = '{}/cnn_lv2_fold{}.pth'.format(checkpoint_dir, fold)
    model.load_state_dict(torch.load(CHECKPOINT))
    model.eval()
    return model

def load_lgbm_lv2_checkpoint(checkpoint_dir, fold=0, i=0):
    model = lightgbm.Booster(model_file='{}/lgbm_lv2_{}_fold{}.pth'.format(checkpoint_dir, i, fold))
    return model

In [10]:
image_ids = []

In [11]:
eb6_512_f0 = load_checkpoint(CHECKPOINT_DIR, backbone='eb6', image_size=512, fold=0)
eb6_512_f1 = load_checkpoint(CHECKPOINT_DIR, backbone='eb6', image_size=512, fold=1)
eb6_512_f2 = load_checkpoint(CHECKPOINT_DIR, backbone='eb6', image_size=512, fold=2)
eb6_512_f3 = load_checkpoint(CHECKPOINT_DIR, backbone='eb6', image_size=512, fold=3)
eb6_512_f4 = load_checkpoint(CHECKPOINT_DIR, backbone='eb6', image_size=512, fold=4)

test_dataset = CLDTestDataset(df=TEST_DF, root_dir=TEST_DIR, image_size=512)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=WORKERS)

eb6_512_preds = []
for images, images_cropped, ids in tqdm(test_loader):
    images = images.cuda()
    images_cropped = images_cropped.cuda()
    image_ids.extend(ids)
    pred = []
    with torch.cuda.amp.autocast(), torch.no_grad():
        pred.append(eb6_512_f0(images))
        pred.append(eb6_512_f0(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb6_512_f0(images_cropped))
        
        pred.append(eb6_512_f1(images))
        pred.append(eb6_512_f1(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb6_512_f1(images_cropped))
        
        pred.append(eb6_512_f2(images))
        pred.append(eb6_512_f2(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb6_512_f2(images_cropped))
        
        pred.append(eb6_512_f3(images))
        pred.append(eb6_512_f3(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb6_512_f3(images_cropped))
        
        pred.append(eb6_512_f4(images))
        pred.append(eb6_512_f4(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb6_512_f4(images_cropped))
        
    pred = torch.stack(pred, -1)
    eb6_512_preds.append(pred)

image_ids = np.array(image_ids)
eb6_512_preds = torch.cat(eb6_512_preds).data.cpu().numpy()

del eb6_512_f0
del eb6_512_f1
del eb6_512_f2
del eb6_512_f3
del eb6_512_f4
del test_dataset
del test_loader
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 1/1 [00:03<00:00,  3.10s/it]


0

In [12]:
eb4_640_f0 = load_checkpoint(CHECKPOINT_DIR, backbone='eb4', image_size=640, fold=0)
eb4_640_f1 = load_checkpoint(CHECKPOINT_DIR, backbone='eb4', image_size=640, fold=1)
eb4_640_f2 = load_checkpoint(CHECKPOINT_DIR, backbone='eb4', image_size=640, fold=2)
eb4_640_f3 = load_checkpoint(CHECKPOINT_DIR, backbone='eb4', image_size=640, fold=3)
eb4_640_f4 = load_checkpoint(CHECKPOINT_DIR, backbone='eb4', image_size=640, fold=4)

test_dataset = CLDTestDataset(df=TEST_DF, root_dir=TEST_DIR, image_size=640)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=WORKERS)

eb4_640_preds = []
for images, images_cropped, _ in tqdm(test_loader):
    images = images.cuda()
    images_cropped = images_cropped.cuda()
    pred = []
    with torch.cuda.amp.autocast(), torch.no_grad():
        pred.append(eb4_640_f0(images))
        pred.append(eb4_640_f0(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb4_640_f0(images_cropped))
        
        pred.append(eb4_640_f1(images))
        pred.append(eb4_640_f1(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb4_640_f1(images_cropped))
        
        pred.append(eb4_640_f2(images))
        pred.append(eb4_640_f2(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb4_640_f2(images_cropped))
        
        pred.append(eb4_640_f3(images))
        pred.append(eb4_640_f3(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb4_640_f3(images_cropped))
        
        pred.append(eb4_640_f4(images))
        pred.append(eb4_640_f4(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(eb4_640_f4(images_cropped))
        
    pred = torch.stack(pred, -1)
    eb4_640_preds.append(pred)

eb4_640_preds = torch.cat(eb4_640_preds).data.cpu().numpy()

del eb4_640_f0
del eb4_640_f1
del eb4_640_f2
del eb4_640_f3
del eb4_640_f4
del test_dataset
del test_loader
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 1/1 [00:01<00:00,  1.84s/it]


0

In [13]:
sr101_448_f0 = load_checkpoint(CHECKPOINT_DIR, backbone='sr101', image_size=448, fold=0)
sr101_448_f1 = load_checkpoint(CHECKPOINT_DIR, backbone='sr101', image_size=448, fold=1)
sr101_448_f2 = load_checkpoint(CHECKPOINT_DIR, backbone='sr101', image_size=448, fold=2)
sr101_448_f3 = load_checkpoint(CHECKPOINT_DIR, backbone='sr101', image_size=448, fold=3)
sr101_448_f4 = load_checkpoint(CHECKPOINT_DIR, backbone='sr101', image_size=448, fold=4)

test_dataset = CLDTestDataset(df=TEST_DF, root_dir=TEST_DIR, image_size=448)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=WORKERS)

sr101_448_preds = []
for images, images_cropped, _ in tqdm(test_loader):
    images = images.cuda()
    images_cropped = images_cropped.cuda()
    pred = []
    with torch.cuda.amp.autocast(), torch.no_grad():
        pred.append(sr101_448_f0(images))
        pred.append(sr101_448_f0(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(sr101_448_f0(images_cropped))
        
        pred.append(sr101_448_f1(images))
        pred.append(sr101_448_f1(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(sr101_448_f1(images_cropped))
        
        pred.append(sr101_448_f2(images))
        pred.append(sr101_448_f2(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(sr101_448_f2(images_cropped))
        
        pred.append(sr101_448_f3(images))
        pred.append(sr101_448_f3(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(sr101_448_f3(images_cropped))
        
        pred.append(sr101_448_f4(images))
        pred.append(sr101_448_f4(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(sr101_448_f4(images_cropped))
        
    pred = torch.stack(pred, -1)
    sr101_448_preds.append(pred)

sr101_448_preds = torch.cat(sr101_448_preds).data.cpu().numpy()

del sr101_448_f0
del sr101_448_f1
del sr101_448_f2
del sr101_448_f3
del sr101_448_f4
del test_dataset
del test_loader
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


0

In [14]:
vitb16_384_f0 = load_checkpoint(CHECKPOINT_DIR, backbone='vitb16', image_size=384, fold=0)
vitb16_384_f1 = load_checkpoint(CHECKPOINT_DIR, backbone='vitb16', image_size=384, fold=1)
vitb16_384_f2 = load_checkpoint(CHECKPOINT_DIR, backbone='vitb16', image_size=384, fold=2)
vitb16_384_f3 = load_checkpoint(CHECKPOINT_DIR, backbone='vitb16', image_size=384, fold=3)
vitb16_384_f4 = load_checkpoint(CHECKPOINT_DIR, backbone='vitb16', image_size=384, fold=4)

test_dataset = CLDTestDataset(df=TEST_DF, root_dir=TEST_DIR, image_size=384)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=WORKERS)

vitb16_384_preds = []
for images, images_cropped, _ in tqdm(test_loader):
    images = images.cuda()
    images_cropped = images_cropped.cuda()
    pred = []
    with torch.cuda.amp.autocast(), torch.no_grad():
        pred.append(vitb16_384_f0(images))
        pred.append(vitb16_384_f0(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(vitb16_384_f0(images_cropped))
        
        pred.append(vitb16_384_f1(images))
        pred.append(vitb16_384_f1(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(vitb16_384_f1(images_cropped))
        
        pred.append(vitb16_384_f2(images))
        pred.append(vitb16_384_f2(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(vitb16_384_f2(images_cropped))
        
        pred.append(vitb16_384_f3(images))
        pred.append(vitb16_384_f3(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(vitb16_384_f3(images_cropped))
        
        pred.append(vitb16_384_f4(images))
        pred.append(vitb16_384_f4(torch.flip(images, dims=(3,)).contiguous()))
        pred.append(vitb16_384_f4(images_cropped))
        
    pred = torch.stack(pred, -1)
    vitb16_384_preds.append(pred)

vitb16_384_preds = torch.cat(vitb16_384_preds).data.cpu().numpy()

del vitb16_384_f0
del vitb16_384_f1
del vitb16_384_f2
del vitb16_384_f3
del vitb16_384_f4
del test_dataset
del test_loader
torch.cuda.empty_cache()
gc.collect()

100%|██████████| 1/1 [00:00<00:00,  1.73it/s]


0

In [15]:
dung_preds = np.zeros((len(TEST_DF), 5), dtype=np.float64)
for fold in range(5):
    cnn_x_test = np.stack([eb6_512_preds[:,:,3*fold:3*fold+3], 
                       eb4_640_preds[:,:,3*fold:3*fold+3], 
                       sr101_448_preds[:,:,3*fold:3*fold+3], 
                       vitb16_384_preds[:,:,3*fold:3*fold+3]], axis=-1)

    cnn_x_test = torch.from_numpy(cnn_x_test).permute(0,3,1,2).cuda()
 
    cnn_lv2_model = load_cnn_lv2_checkpoint(CHECKPOINT_DIR, num_models=4, fold=fold)
    with torch.cuda.amp.autocast(), torch.no_grad():
        cnn_preds = cnn_lv2_model(cnn_x_test)
        cnn_preds = F.softmax(cnn_preds, 1).data.cpu().numpy()
    dung_preds += cnn_preds
    
    del cnn_lv2_model
    del cnn_x_test
dung_preds /= 5.0

del eb6_512_preds
del eb4_640_preds
del sr101_448_preds
del vitb16_384_preds
torch.cuda.empty_cache()
gc.collect()

20

In [16]:
############## TuanHo 0.903 ##############

In [17]:
!pip install ../input/cassavav01/pytorch-image-models > /dev/null

In [18]:
!cp -r ../input/cassavav01/utils .

In [19]:
from collections import defaultdict
import copy
import random
import numpy as np
import os
import shutil
from urllib.request import urlretrieve
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
cudnn.benchmark = True
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import timm
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy, JsdCrossEntropy
from utils import Mixup, RandAugment
from PIL import Image
SEED = 42

In [20]:
def seed_everything(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(SEED)

In [21]:
root = os.path.join(os.environ["HOME"], "/kaggle/input/cassava-leaf-disease-classification")
os.listdir(root)

['train_tfrecords',
 'sample_submission.csv',
 'test_tfrecords',
 'label_num_to_disease_map.json',
 'train_images',
 'train.csv',
 'test_images']

In [22]:
train = pd.read_csv(f'{root}/train.csv')
test = pd.read_csv(f'{root}/sample_submission.csv')
label_map = pd.read_json(f'{root}/label_num_to_disease_map.json', 
                         orient='index')
display(train.head())
display(test.head())
display(label_map)

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


,image_id,label
0,2216849948.jpg,4


,0
0,Cassava Bacterial Blight (CBB)
1,Cassava Brown Streak Disease (CBSD)
2,Cassava Green Mottle (CGM)
3,Cassava Mosaic Disease (CMD)
4,Healthy


In [23]:
models_name = ["resnest26d", "resnest50d" ,"tf_efficientnet_b3_ns", "vit_base_patch16_384"]

In [24]:
WEIGHTS = [

    "../input/cassavav05/resnest26d_fold1_best_epoch_7_final_2nd.pth",
    "../input/cassavav05/resnest26d_fold2_best_epoch_4_final_2nd.pth",
    #
    "../input/cassavav03/tf_efficientnet_b3_ns_fold1_best_epoch_1_final_512.pth",
    #
#     "../input/cassavav06/resnest50d_fold0_best_epoch_13_final_2nd.pth",
    "../input/cassavav07/resnest50d_fold0_best_epoch_10_final_3rd.pth",
#     "../input/cassavav08/resnest50d_fold0_best_epoch_16_final_4th.pth",
    "../input/cassavav06/resnest50d_fold1_best_epoch_17_final_2nd.pth",
    "../input/cassavav09/resnest50d_fold1_best_epoch_8_final_5th_pseudo.pth",
    "../input/cassavav06/resnest50d_fold2_best_epoch_22_final_2nd.pth",
#     "../input/cassavav06/resnest50d_fold3_best_epoch_2_final_2nd.pth",
    "../input/cassavav07/resnest50d_fold3_best_epoch_1_final_3rd.pth",
#     "../input/cassavav08/resnest50d_fold3_best_epoch_2_final_4th.pth",
#     "../input/cassavav06/resnest50d_fold4_best_epoch_10_final_2nd.pth",
    "../input/cassavav06/resnest50d_fold4_best_epoch_15_final_3rd.pth",
    "../input/cassavav08/resnest50d_fold4_best_epoch_10_final-4th.pth",
    "../input/cassavav09/resnest50d_fold4_best_epoch_1_final_5th_pseudo.pth",
]
model_index = 1
ckpt_index = 0

ensemble_models_name = list(2*["resnest26d"] + ["tf_efficientnet_b3_ns"] + 8*["resnest50d"])
ensemble_ckpt_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

params = {
    "visualize": False,
    "debug":False,
    "fold": [0,1,2,3,4],
    "load_pretrained": True,
    "image_size": 512,
    "num_classes": 5,
    "model": models_name[model_index],
    "device": "cuda",
    "batch_size": 16,
    "num_workers": 2,
    "drop_block": 0.2,
    "drop_rate": 0.2,
    "tta": True,
    "kfold_pred":True,
    "error_fix":False,
    "ensemble":True
}

In [25]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None, valid_test=False):
        self.df = df
        self.file_names = df['image_id'].values
        self.transform = transform
        self.valid_test = valid_test
        if self.valid_test:
            self.labels = df['label'].values  
        else:
            assert ValueError("Test data does not have annotation, plz check!")
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{root}/test_images/{file_name}'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if isinstance(self.transform, list):
            outputs = {'images':[],
                       'labels':[]}
             #image0 = transforms.ToPILImage()(image)
             #image0 = self.transform[0](image0)

            for trans in self.transform:
                augmented = trans(image=image)
                image_aug = augmented['image']
                outputs["images"].append(image_aug)
                del image_aug
                
            if self.valid_test:
                label = torch.tensor(self.labels[idx]).long()
                outputs['labels'] = len(self.transform)*[label]
            else:
                outputs['labels'] = len(self.transform)*[-1]
                
            return outputs
        else:
            augmented = self.transform(image=image)
            image = augmented['image'] 
        return image

In [26]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.
        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

In [27]:
def declare_model(name, index=None):
    model = timm.create_model(name,
            pretrained=False,
            num_classes=params["num_classes"],
            drop_rate=params["drop_rate"])
    model = model.to(params["device"])
    model = torch.nn.DataParallel(model) 
        
    if params["load_pretrained"]:
        if index == None: 
            return model
        state_dict = torch.load(WEIGHTS[index])
        print(f"Load pretrained model: {name} ",state_dict["preds"])
        model.load_state_dict(state_dict["model"])
        best_acc = state_dict["preds"]   
    return model
if not params["kfold_pred"] and not params["ensemble"]:
    model = declare_model(params["model"], ckpt_index)

In [28]:
transform_tta0 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),   
     ToTensorV2()
    ]
)

transform_tta1 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.HorizontalFlip(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),   
     ToTensorV2()
    ]
)
transform_tta2 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),        
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.VerticalFlip(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
     ToTensorV2(),
    ]
)
transform_tta3 = A.Compose(
    [
     A.CenterCrop(height=params["image_size"], width=params["image_size"], p=1),
     A.Resize(height=params["image_size"], width=params["image_size"], p=1),
     A.RandomRotate90(p=1.),
     A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),     
     ToTensorV2(),
    ]
)
##### Test TTA with Five Crops
transform_crop_tta0 = T.Compose([
                T.FiveCrop(params["image_size"]),
                T.Lambda(lambda crops: ([T.ToTensor()(crop) for crop in crops])),
                T.Lambda(lambda norms: torch.stack([T.Normalize(mean=[0.5], std=[0.5])(norm) for norm in norms]))
        ]
)
test_transform_tta = [transform_tta0, transform_tta1, transform_tta2, transform_tta3]

#debug
if params['debug']:
    print("In debug mode")
    test = pd.concat([test,test, test])

if params["tta"]:
    test_pred_dataset = TestDataset(test, transform=test_transform_tta)
else:
    test_pred_dataset = TestDataset(test, transform=test_transform_tta[0])
    
test_pred_loader = DataLoader(
    test_pred_dataset, batch_size=params["batch_size"], shuffle=False, num_workers=params['num_workers'], pin_memory=True,
)

In [29]:
test.head()

,image_id,label
0,2216849948.jpg,4


In [30]:
def visualize_tta(data, pred):     
    unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
    if params["tta"]:
        figure, ax = plt.subplots(nrows=1, ncols=num_tta, figsize=(12, 6))
        for i, image in enumerate(data["images"]):
            image = (unorm(image[0]).cpu().numpy()*255).astype(int)
            ax.ravel()[i].imshow(image.transpose(2,1,0))
            ax.ravel()[i].set_title(str(pred), color='GREEN')
            ax.ravel()[i].set_axis_off()
        plt.tight_layout()
        plt.show() 
    else:
        image = (unorm(data).cpu().numpy()*255).astype(int)
        imgplot = plt.imshow(image[0].transpose(2,1,0))
        imgplot = plt.title(str(pred), color='GREEN')
        plt.show()

In [31]:
def gmean(input_x, dim):
    log_x = torch.log(input_x)
    return torch.exp(torch.mean(log_x, dim=dim))

In [32]:
num_tta = len(test_transform_tta)
def tta_predict(loader, params, valid_test=False):
    outputs = []
    preds = []
    models = []
    for ckpt_idx in params["fold"]:
        model = declare_model(params["model"], ckpt_idx)
        model.eval()
        models.append(model)
    #    
    stream = tqdm(loader)    
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            if params["visualize"]:
                visualize_tta(data, pred)
            tta_output = []   
            for i, image in enumerate(data["images"]):
                kfout = [torch.softmax(model(image), dim=1) for model in models]
                tta_output.extend(torch.stack(kfout, dim=0))
            tta_output = torch.stack(tta_output, dim=0).mean(dim=0)
            
            pred = torch.argmax(tta_output, dim=1).cpu().numpy()    
            if params["error_fix"]:
                topk_output, topk_ids = torch.topk(output, params["num_classes"])
                for i in range(len(pred)):
                    ## adjust the output prediction
                    max_1st = topk_ids[i][0]
                    max_2nd = topk_ids[i][1]
                    if  max_1st == 0 and max_2nd == 4 and output[i][max_2nd] > 0.2:
                        pred[i] = max_2nd
                    if max_1st == 3 and max_2nd == 2 and output[i][max_2nd] > 0.2:
                        pred[i] = max_2nd

            preds.extend(pred)
            ## For visualize the TTA 
            if params["visualize"]:
                visualize_tta(data, pred)
    return preds

In [33]:
# Ensemble CV w/ fold predict on validation set
def tta_pred_ensemble_model(loader, params, valid_test=False):
    stream = tqdm(loader)
    models = []
    tuan_preds = []
    print(f"Ensemble below models: {ensemble_models_name}")
    for name, ckpt in zip(ensemble_models_name, ensemble_ckpt_index):
        print(ckpt)
        m = declare_model(name, ckpt)  
        models.append(m.eval())
        del m
    
    with torch.no_grad():
        for i, data in enumerate(stream, start=1):
            tta_output = []   
            for i, image in enumerate(data["images"]):
                kfout = [torch.softmax(model(image), dim=1) for model in models]
                tta_output.extend(torch.stack(kfout, dim=0))
            tta_output = torch.stack(tta_output, dim=0).mean(dim=0)
            
            tta_output = F.softmax(tta_output, 1).data.cpu().numpy()
            
            tuan_preds.append(tta_output)
            
    tuan_preds = np.vstack(tuan_preds).astype(np.float64)
    return tuan_preds

In [34]:
tuan_preds = tta_pred_ensemble_model(test_pred_loader, params)

  0%|          | 0/1 [00:00<?, ?it/s]

Ensemble below models: ['resnest26d', 'resnest26d', 'tf_efficientnet_b3_ns', 'resnest50d', 'resnest50d', 'resnest50d', 'resnest50d', 'resnest50d', 'resnest50d', 'resnest50d', 'resnest50d']
0
Load pretrained model: resnest26d  0.8997
1
Load pretrained model: resnest26d  0.8952
2
Load pretrained model: tf_efficientnet_b3_ns  0.897
3
Load pretrained model: resnest50d  0.8864
4
Load pretrained model: resnest50d  0.8993
5
Load pretrained model: resnest50d  0.9005
6
Load pretrained model: resnest50d  0.8953
7
Load pretrained model: resnest50d  0.8906
8
Load pretrained model: resnest50d  0.8916
9
Load pretrained model: resnest50d  0.8939
10
Load pretrained model: resnest50d  0.892


100%|██████████| 1/1 [00:21<00:00, 21.13s/it]


In [35]:
preds = 0.5*dung_preds + 0.5*tuan_preds
preds = np.argmax(preds, axis=1)

In [36]:
sub_df = pd.DataFrame()
sub_df['image_id'] = np.array(image_ids)
sub_df['label'] = np.array(preds, dtype=int)
sub_df.to_csv('submission.csv', index=False)
print(sub_df.head())

         image_id  label
0  2216849948.jpg      4
